# Part 2 - Run simulations with constant stagnation point

### First load some references

In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases.PrintingNip;
Init();

### Init Database etc.

In [ ]:
string ProjectName4Correlation = "SFB1194_K65_Part1";
string ProjectName = "SFB1194_K65_Part2";

In [ ]:
static var myBatch = BoSSSshell.GetDefaultQueue();

In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName4Correlation);
static var myDb4Correlation = BoSSSshell.OpenDatabase(BoSSSshell.WorkflowMgm.DefaultDatabase.Path);
Console.WriteLine("Initializing : " + myDb4Correlation.Path); // we need to init the static var! otherwise we end up with a reference to the wrong database!

In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName);
static var myDb = BoSSSshell.OpenDatabase(BoSSSshell.WorkflowMgm.DefaultDatabase.Path);
Console.WriteLine("Initializing : " + myDb.Path);

In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

### Construct parameterized velocity splines

In [ ]:
double H0 = 1e-5;
double V0 = 1.0;
double P0 = 1e6;

In [ ]:
var sessions = myDb4Correlation.Sessions.Where(s => s.ProjectName == "SFB1194_K65_Part1").ToArray();
var sV = sessions.Single(s => Convert.ToDouble(s.KeysAndQueries["id:delta"]).ApproxEqual(H0) &  Convert.ToDouble(s.KeysAndQueries["id:P_Diff"]).ApproxEqual(0.0) & Convert.ToDouble(s.KeysAndQueries["id:V_Wall"]).ApproxEqual(V0));
var sP = sessions.Single(s => Convert.ToDouble(s.KeysAndQueries["id:delta"]).ApproxEqual(H0) &  Convert.ToDouble(s.KeysAndQueries["id:P_Diff"]).ApproxEqual(P0) & Convert.ToDouble(s.KeysAndQueries["id:V_Wall"]).ApproxEqual(0.0));

Function to create the velocity profile along the symmetry axis

In [ ]:
using MathNet.Numerics.Interpolation;

In [ ]:
static public CubicSpline ConstructVelocitySpline(ISessionInfo si) {
    DGField VelocityX = si.Timesteps.Last().Fields.Single(f => f.Identification == "VelocityX");
    var grd = (GridData)VelocityX.GridDat;
    EdgeMask em = new EdgeMask(grd, X => Math.Abs(X[1]) < 1e-12); // symmetry axis

    var Spline = Postprocessing.SplineOnEdge(em, VelocityX, 0, out double lB, out double uB);
    return Spline;
}

Evaluate the profile for both simulations and superpose

In [ ]:
var SplineV = ConstructVelocitySpline(sV);
var SplineP = ConstructVelocitySpline(sP);

In [ ]:
double[] X = GenericBlas.Linspace(-0.01, 0.01, 1000);
double[] VV = X.Select(x => SplineV.Interpolate(x)).ToArray();
double[] VP = X.Select(x => SplineP.Interpolate(x)).ToArray();

BoSSSshell.Plot(X, VV, "u_A", "k-", X, VP, "u_B", "k--")

In [ ]:
Func<double, double, double, double, double> VelocityFunc = (x, h, v, p) => v/V0 * SplineV.Interpolate(x * Math.Sqrt(H0) / Math.Sqrt(h)) + (p * Math.Pow(h,1.5))/(P0 * Math.Pow(H0,1.5)) * SplineP.Interpolate(x * Math.Sqrt(H0) / Math.Sqrt(h));

Set Velocity to be zero at a certain x_stag and calculate the necessary pressure difference

In [ ]:
Func<double, double, double, double> PressureFunc = (x_stag, h, v) => -v/V0 * SplineV.Interpolate(x_stag * Math.Sqrt(H0) / Math.Sqrt(h)) * (P0 * Math.Pow(H0,1.5)) / (Math.Pow(h,1.5) * SplineP.Interpolate(x_stag * Math.Sqrt(H0) / Math.Sqrt(h)));

### Setup Simulations

In [ ]:
double[] deltaS = new double[] {
    0.001, // millimeters
    //0.0005, 
    0.0001, 
    //0.00005, 
    0.00001, 
    //0.000005, 
    0.000001 }; // micro-meter
double[] V = new double[] {
  0.1, 
  0.5, 
  1.0,
  5.0, 
  10.0}; // Wall velocities in m/s

// always same
int Res = 10;
int DGdegree = 5;
double R = 0.1;
double x_stag = 0.01; // always at 10 mm

// set grid to be saved in database
GridFactory.myDb = myDb;

In [ ]:
var controls = new List<XNSE_Control>();

foreach(double delta in deltaS) {
   foreach(double V_wall in V) {

      double p = PressureFunc(x_stag, delta, V_wall);

      var C = new XNSE_Control();
      C.SetDGdegree(DGdegree);
      C.SetGrid(GridFactory.GenerateGrid(Res,delta, R));
      C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Res", Res));
      C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("delta", delta));
      C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Radius", R));
      C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("V_Wall", V_wall));
      C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("P_Diff", p));


      C.AddBoundaryValue("wall_walze", "VelocityX", BoundaryValueFactory.Get_VelX(delta, V_wall, R));
      C.AddBoundaryValue("wall_walze", "VelocityY", BoundaryValueFactory.Get_VelY(delta, V_wall, R));
      C.AddBoundaryValue("wall_substrat", "VelocityX", BoundaryValueFactory.Get_VelX(delta, V_wall, R));
      C.AddBoundaryValue("wall_substrat", "VelocityY", BoundaryValueFactory.Get_VelY(delta, V_wall, R));
      C.AddBoundaryValue("pressure_outlet_in", "Pressure", $"X => {p}", false);
      C.AddBoundaryValue("pressure_outlet_out", "Pressure", $"X => {-p}", false);


      C.TimesteppingMode = AppControl._TimesteppingMode.Steady;

      C.PhysicalParameters.rho_A             = 1026.4;
      C.PhysicalParameters.mu_A              = 0.0395;
      C.PhysicalParameters.IncludeConvection = false;
      C.Timestepper_LevelSetHandling         = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
      
      C.SessionName = "J" + (20*Res*Res) + "_delta" + delta + "_V" + V_wall + "_P" + p;     
      controls.Add(C);
   }
}

In [ ]:
Console.WriteLine(controls.Count);

### Run Simulations

Workaround so we need to deploy the rather large executables only once!

In [ ]:
string mngdir = "SFB1194K65_Part2_" + DateTime.Now.ToString("MMMdd_HHmmss") + "_managed";
DirectoryInfo ManagedOverride = new DirectoryInfo(Path.Combine(myBatch.DeploymentBaseDirectory, mngdir));
ManagedOverride.Create();
typeof(XNSE<XNSE_Control>).Assembly.DeployAt(ManagedOverride);
string RelManagedPath = "../" + mngdir + "/" + Path.GetFileName(typeof(XNSE<XNSE_Control>).Assembly.Location);

In [ ]:
string ntvdir = "SFB1194K65_Part2_" + DateTime.Now.ToString("MMMdd_HHmmss") + "_amd64";
DirectoryInfo NativeOverride = new DirectoryInfo(Path.Combine(myBatch.DeploymentBaseDirectory, ntvdir));
NativeOverride.Create();
MetaJobMgrIO.CopyDirectoryRec(ilPSP.Environment.NativeLibraryDir, NativeOverride.FullName, null);
myBatch.DeployRuntime = false; // we did this manually!

In [ ]:
foreach(var c in controls) {
    Job j = c.CreateJob();
    j.EnvironmentVars.Add(BoSSS.Foundation.IO.Utils.BOSSS_NATIVE_OVERRIDE, NativeOverride.FullName);
    j.EntryAssemblyRedirection = RelManagedPath;
    j.Activate(myBatch);
}

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(18000);